# Corona Cases

This notebook contains a bar chart to visualize the corona cases either per country of worldwide. The underlying data is provided by [Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19/). 

## Import packages

- After the initial run, comment this line to minimize the execution time
```bash
#!jupyter nbextension enable --py widgetsnbextension
```

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

## Prepare dataset

### Data cleansing

- Replaces NA/NaN values
- Drops not required columns (Province/State, Lat, Long)
- Aggregates the corona cases per country
- Calculates the worldwide corona cases
- Adds a status per case (Confirmed, Death, Recovered)

In [ ]:
def data_cleansing(df, status):
    df=df.fillna("")
    df=df.drop(["Province/State", "Lat", "Long"], axis=1)
    df=df.rename(columns={"Country/Region": "Country"})
    df=df.groupby(["Country"], sort=True).sum() # Country will become the new index
    df.loc["Worldwide"] = df.sum(numeric_only=True)
    df.insert(0, "Status", status, allow_duplicates=True)
    df=df.reset_index().set_index(["Country", "Status"])
    return df

### Load confirmed cases

In [ ]:
# load confirmed corona cases
df_confirmed=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df_confirmed=data_cleansing(df_confirmed, "Confirmed")
df_confirmed.head(3)

### Load death cases

In [ ]:
# load death corona cases
df_death=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv")
df_death=data_cleansing(df_death, "Death")
df_death.head(3)

### Load recovered cases

In [ ]:
# load recovered corona cases
df_recovered=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")
df_recovered=data_cleansing(df_recovered, "Recovered")
df_recovered.head(3)

### Merge all cases

In [ ]:
df=pd.concat([df_confirmed,df_death,df_recovered]).sort_values(by=["Country", "Status"])
df.head(3)

## Visualize dataset

In [ ]:
# Bar chart displaying corona cases
days=df.columns
country="Worldwide"
confirmed=df.loc[country,"Confirmed"]
death=df.loc[country,"Death"]
recovered=df.loc[country,"Recovered"]
fig = go.FigureWidget(data=[
                    go.Bar(name="Confirmed", x=days, y=confirmed, marker_color="red"),
                    go.Bar(name="Death", x=days, y=death, marker_color="black"),
                    go.Bar(name="Recovered", x=days, y=recovered, marker_color="green"),
                ],
                layout=go.Layout(plot_bgcolor = "#EEEEEE"))

# Dropdown to select country to be displayed in bar chart
countries=df.index.get_level_values(0).unique().tolist()
dropdown = widgets.Dropdown(
    description='Corona cases:',
    value=country,
    options=countries,
    style={'description_width': 'initial'}
)
def response(change):
    country=dropdown.value
    with fig.batch_update():
            fig.data[0].y = df.loc[country,"Confirmed"]
            fig.data[1].y = df.loc[country,"Death"]
            fig.data[2].y = df.loc[country,"Recovered"]
dropdown.observe(response, names="value")

# Display dropdown and figure
widgets.VBox([dropdown, fig])